In [5]:
import tensorflow as tf
import pandas as pd

from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import os

In [14]:
le = preprocessing.LabelEncoder()

accuracy1_data=pd.read_csv("../Datasets/accuracy2.csv")

accuracy1_data=accuracy1_data.fillna(value=-150)

# Create feature and target arrays
y_tmp=accuracy1_data['location']
y=le.fit_transform(y_tmp)
X=accuracy1_data
X=X.drop(['location'], axis = 1)
X=(X-X.mean())/X.std()
X+=3
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,stratify=y)


X_train.head()

,00:41:D2:C1:C1:32,88:F0:77:FD:F9:00,00:41:D2:C1:C1:30,00:41:D2:C1:C1:39,88:F0:77:FD:F9:09,C2:2B:F9:8D:63:ED,88:F0:77:FD:AB:20,00:DA:55:72:48:82,00:DA:55:51:CC:B9,00:DA:55:72:48:89,...,88:F0:77:FD:CD:12,88:F0:77:FD:E3:00,88:F0:77:FD:A8:90,00:DA:55:51:CC:B0,00:DA:55:72:48:80,78:72:5D:8E:C4:C2,78:72:5D:8E:C4:C0,78:72:5D:8E:C4:C9,00:DA:55:72:43:89,88:F0:77:FD:FF:59
73,1.634256,2.707817,1.94545,1.845976,2.707875,5.296872,2.813223,2.685845,2.708393,2.81337,...,2.84837,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934
77,1.634256,2.707817,1.94545,1.845976,2.707875,5.591601,2.813223,2.685845,2.708393,2.81337,...,2.84837,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934
66,3.675290,2.707817,3.88397,3.820358,2.707875,2.054860,2.813223,2.685845,2.708393,2.81337,...,2.84837,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934
8,1.634256,2.707817,1.94545,1.845976,2.707875,2.526425,2.813223,2.685845,2.708393,2.81337,...,2.84837,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934
32,1.634256,2.707817,1.94545,1.845976,2.707875,2.231697,2.813223,2.685845,2.708393,2.81337,...,2.84837,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934,2.8934


In [15]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)

#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score'] #naming the dataframe columns

In [ ]:
my_features=list(featureScores.nlargest(20,'Score').Specs)

X_train=X_train[my_features]
X_test=X_test[my_features]

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(len(X_train.columns)),
                                
                                    tf.keras.layers.Dense(15, activation='tanh'),

                                    tf.keras.layers.Dense(12),

                                    tf.keras.layers.Dense(10, activation='softmax')])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
early_stop_callback= tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', min_delta=0, patience=20, verbose=1,
    mode='max', baseline=None, restore_best_weights=True
)

In [ ]:
os.makedirs('tmp', exist_ok = True)

checkpoint_filepath = 'tmp/checkpoint-{epoch:03d}-{accuracy:.4f}.hdf5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(X_train,y_train,epochs=250,batch_size=28, callbacks=[early_stop_callback,checkpoint_callback])

Epoch 1/250
3/3 [==============================] - 0s 6ms/step - loss: 2.4786 - accuracy: 0.1667
Epoch 2/250
3/3 [==============================] - 0s 5ms/step - loss: 2.4140 - accuracy: 0.1818
Epoch 3/250
3/3 [==============================] - 0s 4ms/step - loss: 2.3567 - accuracy: 0.2121
Epoch 4/250
3/3 [==============================] - 0s 6ms/step - loss: 2.3139 - accuracy: 0.1970
Epoch 5/250
3/3 [==============================] - 0s 7ms/step - loss: 2.2809 - accuracy: 0.1818
Epoch 6/250
3/3 [==============================] - 0s 4ms/step - loss: 2.2554 - accuracy: 0.1515
Epoch 7/250
3/3 [==============================] - 0s 4ms/step - loss: 2.2304 - accuracy: 0.1818
Epoch 8/250
3/3 [==============================] - 0s 3ms/step - loss: 2.2144 - accuracy: 0.1818
Epoch 9/250
3/3 [==============================] - 0s 4ms/step - loss: 2.1971 - accuracy: 0.1667
Epoch 10/250
3/3 [==============================] - 0s 3ms/step - loss: 2.1836 - accuracy: 0.1667
Epoch 11/250
3/3 [===========

In [ ]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 107ms/step - loss: 1.6456 - accuracy: 0.3182


[1.645643949508667, 0.3181818127632141]

In [ ]:
#model.save('accuracy2-45.hdf5')